### Prepare basin GRU (hydrologic unit or GRU) and flowline shapefiles ###

#### If these don't both pre-exist, run this script before all other scripts ####

This script includes:<br>
1a. if needed, extract basin GRU shapefile from a large-domain GRU shapefile.<br> 
1b. write basin gruId.txt list
2. extract basin flowlines shapefile from a large-domain flowlines shapefile.<br>
3. reproject basin GRU and flowlines shapefiles to a common equal coordinate system. <br>
The script also makes some directories used in the discretization process.

In [1]:
# import libraries
import os, sys
sys.path.append('../')
import functions.geospatial_analysis as ga
import functions.utils as ut
import geopandas as gpd
import rasterio as rio
from rasterio.warp import Resampling
import functions.ogr2ogr as ogr2ogr
import numpy as np

#### Set up paths, filenames, directories ####

In [2]:
ut.start_logging()

# common paths
control_file    = '/Users/drc858/GitHub/watershed_tools/test_cases/tuolumne/control_tuolumne.txt'

#config          = ut.read_complete_control_file(control_file, logging=True)
basin_data_path = ut.read_from_control(control_file, 'basin_data_path')
basin_name      = ut.read_from_control(control_file, 'basin_name')

2023-04-19 16:03:54,747 - utils.py - INFO - Log File Generated: /Users/drc858/GitHub/watershed_tools/functions/./../logs/log_file_2023-04-19_16:03:54.log
2023-04-19 16:03:54,750 - utils.py - INFO - Log File Generated: /Users/drc858/GitHub/watershed_tools/functions/./../logs/log_file_2023-04-19_16:03:54.log
2023-04-19 16:03:54,751 - utils.py - DEBUG - Set_filename() was not successful for file basin_name
2023-04-19 16:03:54,752 - utils.py - INFO - Setting: basin_name = Tuolumne
2023-04-19 16:03:54,753 - utils.py - DEBUG - Set_filename() was not successful for file basin_data_path
2023-04-19 16:03:54,754 - utils.py - INFO - Setting: basin_data_path = /Users/drc858/GitHub/watershed_tools/test_cases/tuolumne/
2023-04-19 16:03:54,756 - utils.py - INFO - Setting: basin_gru_shp = /Users/drc858/GitHub/watershed_tools/test_cases/tuolumne/gis/tuolumne_base.shp
2023-04-19 16:03:54,757 - utils.py - DEBUG - Set_filename() was not successful for file main_path
2023-04-19 16:03:54,757 - utils.py - IN

2023-04-19 16:03:54,838 - utils.py - INFO - Setting: toGRU_fieldname = NextDownID
2023-04-19 16:03:54,840 - utils.py - DEBUG - Set_filename() was not successful for file gru_fieldname
2023-04-19 16:03:54,841 - utils.py - INFO - Setting: gru_fieldname = COMID
2023-04-19 16:03:54,843 - utils.py - DEBUG - Set_filename() was not successful for file merge_cat_riv_file
2023-04-19 16:03:54,844 - utils.py - INFO - Setting: merge_cat_riv_file = True
2023-04-19 16:03:54,846 - utils.py - DEBUG - Set_filename() was not successful for file cat_file
2023-04-19 16:03:54,846 - utils.py - INFO - Setting: cat_file = /Users/drc858/GitHub/watershed_tools/test_cases/tuolumne/gis/cat_pfaf_77_MERIT_Hydro_v07_Basins_v01_bugfix1.shp
2023-04-19 16:03:54,849 - utils.py - DEBUG - Set_filename() was not successful for file riv_file
2023-04-19 16:03:54,849 - utils.py - INFO - Setting: riv_file = /Users/drc858/GitHub/watershed_tools/test_cases/tuolumne/gis/riv_pfaf_77_MERIT_Hydro_v07_Basins_v01_bugfix1.shp
2023-04-1

In [3]:
# make standard directories
if not os.path.exists(basin_data_path):
    os.makedirs(basin_data_path)
plot_path  = os.path.join(basin_data_path, 'plots/')
if not os.path.exists(plot_path):
    os.makedirs(plot_path)
gis_path  = os.path.join(basin_data_path, 'gis/')
if not os.path.exists(gis_path):
    os.makedirs(gis_path)

In [4]:
# projection system
new_epsg = ut.read_from_control(control_file, 'epsg') 
dest_crs = rio.crs.CRS.from_epsg(new_epsg)

In [5]:
# set basin shapefiles
basin_gru_shp           = ut.set_filename(control_file, 'basin_gru_shp')  # may exist
basin_flowlines_shp     = ut.set_filename(control_file, 'basin_flowlines_shp') # may exist; is always _prj

# gru fieldname and text file
gru_fieldname           = ut.read_from_control(control_file, 'gru_fieldname')      
basin_gruId_txt         = ut.set_filename(control_file, 'basin_gruId_txt')

# derived filenames
basin_gru_prj_shp       = basin_gru_shp.split('.shp')[0]+'_prj.shp'

if not os.path.exists(basin_gru_shp):
    print('WARNING: basin_shapefile not found.  If needed, check path/filename before proceeding:', basin_gru_shp)

#### Set basin GRU shapefile (extract from larger full-domain if needed) ####

In [6]:
# if the basin shapefile doesn't exist, it needs to be extracted from another larger GRU shapefile
if not os.path.exists(basin_gru_shp):

    # ---- extract basin GRU shapefile and ID list from a larger full-domain GRU shapefile ---- 

    # read filename and other necessary info
    fulldom_gru_shp   = ut.read_from_control(control_file, 'fulldom_gru_shpfile')
    outlet_gruId      = ut.read_from_control(control_file, 'basin_outlet_gruId')
    toGRU_fieldname   = ut.read_from_control(control_file, 'toGRU_fieldname')
    data              = gpd.read_file(fulldom_gru_shp)
    
    # check whether two useful columns (gru_field, toGRU_field) are in gru_shp.
    if not gru_fieldname in data.columns.values:
        exit(gru_fieldname + ' column does not exist in shapefile.')
    else:
        grus = data[gru_fieldname].values
    if not toGRU_fieldname in data.columns.values:
        exit(toGRU_fieldname + ' column does not exist in shapefile.')
    else:
        togrus = data[toGRU_fieldname].values
    # extract only the useful columns to save data memory.
    data = data[[gru_fieldname, toGRU_fieldname, 'geometry']] 

    # ---- search upstream GRUs ---- 
    # method 1: search upstream grus base on the most downstream gruId
    upstream_grus = [outlet_gruId]           # list of upstream grus. initiate with outlet_gruid
    gru_found     = np.unique(grus[np.where(togrus==outlet_gruId)]) # find all the upstream grus that drain to outlet_gruid.
    upstream_grus.extend(list(gru_found))    # add the found upstream grus of outlet_gruid to upstream_grus list
    round_num     = 0                        # record the round number of searching.

    while len(gru_found) != 0: # terminate searching upstream grus until no one can be found any more.
        round_num = round_num+1
        print("Round %d: %d GRUs found." % (round_num, len(upstream_grus)))

        # search upstream grus
        gru_found_next = []
        for gru_i in gru_found:
            gru_found_next.extend(list(grus[np.where(togrus==gru_i)]))
        gru_found_next = unique(gru_found_next)

        # identify if the found GRUs exist in upstrm_grus
        gru_found = [gru for gru in gru_found_next if not gru in upstream_grus]
        upstream_grus.extend(gru_found)

        # alternate method: manually add upstream_grus when the list of upstream grus is known. 
        #upstream_grus= np.loadtxt('/glade/u/home/andywood/proj/SHARP/wreg/bighorn/prep/lists/gruIds.06279940.txt',dtype=int)

    # ---- save upstream GRU shapefile ---- 
    data[data[gru_fieldname].isin(upstream_grus)].to_file(basin_gru_shp)

2023-04-19 16:03:54,924 - env.py - DEBUG - GDAL data found in package: path='/Users/drc858/.pyenv/versions/3.9.16/envs/watershed_tools/lib/python3.9/site-packages/fiona/gdal_data'.
2023-04-19 16:03:54,925 - env.py - DEBUG - PROJ data found in package: path='/Users/drc858/.pyenv/versions/3.9.16/envs/watershed_tools/lib/python3.9/site-packages/fiona/proj_data'.
2023-04-19 16:03:54,930 - env.py - DEBUG - GDAL data found in package: path='/Users/drc858/.pyenv/versions/3.9.16/envs/watershed_tools/lib/python3.9/site-packages/fiona/gdal_data'.
2023-04-19 16:03:54,931 - env.py - DEBUG - PROJ data found in package: path='/Users/drc858/.pyenv/versions/3.9.16/envs/watershed_tools/lib/python3.9/site-packages/fiona/proj_data'.
2023-04-19 16:03:54,949 - collection.py - DEBUG - Got coordinate system
2023-04-19 16:03:54,952 - collection.py - DEBUG - Got coordinate system
2023-04-19 16:03:54,966 - file.py - DEBUG - Matched. confidence=100, c_code=b'4326', c_name=b'EPSG'
2023-04-19 16:03:54,968 - collec

2023-04-19 16:03:55,123 - geodataframe.py - DEBUG - Next index: 106
2023-04-19 16:03:55,124 - geodataframe.py - DEBUG - Next index: 107
2023-04-19 16:03:55,125 - geodataframe.py - DEBUG - Next index: 108
2023-04-19 16:03:55,126 - geodataframe.py - DEBUG - Next index: 109
2023-04-19 16:03:55,128 - geodataframe.py - DEBUG - Next index: 110
2023-04-19 16:03:55,129 - geodataframe.py - DEBUG - Next index: 111
2023-04-19 16:03:55,131 - geodataframe.py - DEBUG - Next index: 112
2023-04-19 16:03:55,132 - geodataframe.py - DEBUG - Next index: 113
2023-04-19 16:03:55,133 - geodataframe.py - DEBUG - Next index: 114
2023-04-19 16:03:55,135 - geodataframe.py - DEBUG - Next index: 115
2023-04-19 16:03:55,136 - geodataframe.py - DEBUG - Next index: 116
2023-04-19 16:03:55,138 - geodataframe.py - DEBUG - Next index: 117
2023-04-19 16:03:55,139 - geodataframe.py - DEBUG - Next index: 118
2023-04-19 16:03:55,140 - geodataframe.py - DEBUG - Next index: 119
2023-04-19 16:03:55,141 - geodataframe.py - DEBU

2023-04-19 16:03:55,253 - geodataframe.py - DEBUG - Next index: 227
2023-04-19 16:03:55,254 - geodataframe.py - DEBUG - Next index: 228
2023-04-19 16:03:55,255 - geodataframe.py - DEBUG - Next index: 229
2023-04-19 16:03:55,256 - geodataframe.py - DEBUG - Next index: 230
2023-04-19 16:03:55,257 - geodataframe.py - DEBUG - Next index: 231
2023-04-19 16:03:55,258 - geodataframe.py - DEBUG - Next index: 232
2023-04-19 16:03:55,259 - geodataframe.py - DEBUG - Next index: 233
2023-04-19 16:03:55,260 - geodataframe.py - DEBUG - Next index: 234
2023-04-19 16:03:55,261 - geodataframe.py - DEBUG - Next index: 235
2023-04-19 16:03:55,263 - geodataframe.py - DEBUG - Next index: 236
2023-04-19 16:03:55,263 - geodataframe.py - DEBUG - Next index: 237
2023-04-19 16:03:55,265 - geodataframe.py - DEBUG - Next index: 238
2023-04-19 16:03:55,266 - geodataframe.py - DEBUG - Next index: 239
2023-04-19 16:03:55,268 - geodataframe.py - DEBUG - Next index: 240
2023-04-19 16:03:55,269 - geodataframe.py - DEBU

2023-04-19 16:03:55,377 - geodataframe.py - DEBUG - Next index: 348
2023-04-19 16:03:55,378 - geodataframe.py - DEBUG - Next index: 349
2023-04-19 16:03:55,379 - geodataframe.py - DEBUG - Next index: 350
2023-04-19 16:03:55,380 - geodataframe.py - DEBUG - Next index: 351
2023-04-19 16:03:55,381 - geodataframe.py - DEBUG - Next index: 352
2023-04-19 16:03:55,382 - geodataframe.py - DEBUG - Next index: 353
2023-04-19 16:03:55,383 - geodataframe.py - DEBUG - Next index: 354
2023-04-19 16:03:55,385 - geodataframe.py - DEBUG - Next index: 355
2023-04-19 16:03:55,386 - geodataframe.py - DEBUG - Next index: 356
2023-04-19 16:03:55,386 - geodataframe.py - DEBUG - Next index: 357
2023-04-19 16:03:55,387 - geodataframe.py - DEBUG - Next index: 358
2023-04-19 16:03:55,388 - geodataframe.py - DEBUG - Next index: 359
2023-04-19 16:03:55,389 - geodataframe.py - DEBUG - Next index: 360
2023-04-19 16:03:55,390 - geodataframe.py - DEBUG - Next index: 361
2023-04-19 16:03:55,391 - geodataframe.py - DEBU

2023-04-19 16:03:55,511 - geodataframe.py - DEBUG - Next index: 469
2023-04-19 16:03:55,512 - geodataframe.py - DEBUG - Next index: 470
2023-04-19 16:03:55,513 - geodataframe.py - DEBUG - Next index: 471
2023-04-19 16:03:55,514 - geodataframe.py - DEBUG - Next index: 472
2023-04-19 16:03:55,515 - geodataframe.py - DEBUG - Next index: 473
2023-04-19 16:03:55,516 - geodataframe.py - DEBUG - Next index: 474
2023-04-19 16:03:55,517 - geodataframe.py - DEBUG - Next index: 475
2023-04-19 16:03:55,518 - geodataframe.py - DEBUG - Next index: 476
2023-04-19 16:03:55,519 - geodataframe.py - DEBUG - Next index: 477
2023-04-19 16:03:55,520 - geodataframe.py - DEBUG - Next index: 478
2023-04-19 16:03:55,521 - geodataframe.py - DEBUG - Next index: 479
2023-04-19 16:03:55,522 - geodataframe.py - DEBUG - Next index: 480
2023-04-19 16:03:55,523 - geodataframe.py - DEBUG - Next index: 481
2023-04-19 16:03:55,524 - geodataframe.py - DEBUG - Next index: 482
2023-04-19 16:03:55,525 - geodataframe.py - DEBU

...

2023-04-19 16:04:49,109 - geodataframe.py - DEBUG - Next index: 43038
2023-04-19 16:04:49,110 - geodataframe.py - DEBUG - Next index: 43039
2023-04-19 16:04:49,111 - geodataframe.py - DEBUG - Next index: 43040
2023-04-19 16:04:49,112 - geodataframe.py - DEBUG - Next index: 43041
2023-04-19 16:04:49,113 - geodataframe.py - DEBUG - Next index: 43042
2023-04-19 16:04:49,115 - geodataframe.py - DEBUG - Next index: 43043
2023-04-19 16:04:49,116 - geodataframe.py - DEBUG - Next index: 43044
2023-04-19 16:04:49,118 - geodataframe.py - DEBUG - Next index: 43045
2023-04-19 16:04:49,119 - geodataframe.py - DEBUG - Next index: 43046
2023-04-19 16:04:49,121 - geodataframe.py - DEBUG - Next index: 43047
2023-04-19 16:04:49,123 - geodataframe.py - DEBUG - Next index: 43048
2023-04-19 16:04:49,124 - geodataframe.py - DEBUG - Next index: 43049
2023-04-19 16:04:49,125 - geodataframe.py - DEBUG - Next index: 43050
2023-04-19 16:04:49,126 - geodataframe.py - DEBUG - Next index: 43051
2023-04-19 16:04:49,

2023-04-19 16:04:49,377 - geodataframe.py - DEBUG - Next index: 43156
2023-04-19 16:04:49,378 - geodataframe.py - DEBUG - Next index: 43157
2023-04-19 16:04:49,380 - geodataframe.py - DEBUG - Next index: 43158
2023-04-19 16:04:49,381 - geodataframe.py - DEBUG - Next index: 43159
2023-04-19 16:04:49,382 - geodataframe.py - DEBUG - Next index: 43160
2023-04-19 16:04:49,383 - geodataframe.py - DEBUG - Next index: 43161
2023-04-19 16:04:49,385 - geodataframe.py - DEBUG - Next index: 43162
2023-04-19 16:04:49,386 - geodataframe.py - DEBUG - Next index: 43163
2023-04-19 16:04:49,387 - geodataframe.py - DEBUG - Next index: 43164
2023-04-19 16:04:49,388 - geodataframe.py - DEBUG - Next index: 43165
2023-04-19 16:04:49,389 - geodataframe.py - DEBUG - Next index: 43166
2023-04-19 16:04:49,389 - geodataframe.py - DEBUG - Next index: 43167
2023-04-19 16:04:49,390 - geodataframe.py - DEBUG - Next index: 43168
2023-04-19 16:04:49,391 - geodataframe.py - DEBUG - Next index: 43169
2023-04-19 16:04:49,

2023-04-19 16:04:49,503 - geodataframe.py - DEBUG - Next index: 43274
2023-04-19 16:04:49,504 - geodataframe.py - DEBUG - Next index: 43275
2023-04-19 16:04:49,505 - geodataframe.py - DEBUG - Next index: 43276
2023-04-19 16:04:49,506 - geodataframe.py - DEBUG - Next index: 43277
2023-04-19 16:04:49,507 - geodataframe.py - DEBUG - Next index: 43278
2023-04-19 16:04:49,508 - geodataframe.py - DEBUG - Next index: 43279
2023-04-19 16:04:49,509 - geodataframe.py - DEBUG - Next index: 43280
2023-04-19 16:04:49,510 - geodataframe.py - DEBUG - Next index: 43281
2023-04-19 16:04:49,511 - geodataframe.py - DEBUG - Next index: 43282
2023-04-19 16:04:49,512 - geodataframe.py - DEBUG - Next index: 43283
2023-04-19 16:04:49,513 - geodataframe.py - DEBUG - Next index: 43284
2023-04-19 16:04:49,514 - geodataframe.py - DEBUG - Next index: 43285
2023-04-19 16:04:49,515 - geodataframe.py - DEBUG - Next index: 43286
2023-04-19 16:04:49,516 - geodataframe.py - DEBUG - Next index: 43287
2023-04-19 16:04:49,

2023-04-19 16:04:49,620 - geodataframe.py - DEBUG - Next index: 43392
2023-04-19 16:04:49,621 - geodataframe.py - DEBUG - Next index: 43393
2023-04-19 16:04:49,622 - geodataframe.py - DEBUG - Next index: 43394
2023-04-19 16:04:49,623 - geodataframe.py - DEBUG - Next index: 43395
2023-04-19 16:04:49,624 - geodataframe.py - DEBUG - Next index: 43396
2023-04-19 16:04:49,625 - geodataframe.py - DEBUG - Next index: 43397
2023-04-19 16:04:49,625 - geodataframe.py - DEBUG - Next index: 43398
2023-04-19 16:04:49,626 - geodataframe.py - DEBUG - Next index: 43399
2023-04-19 16:04:49,627 - geodataframe.py - DEBUG - Next index: 43400
2023-04-19 16:04:49,628 - geodataframe.py - DEBUG - Next index: 43401
2023-04-19 16:04:49,629 - geodataframe.py - DEBUG - Next index: 43402
2023-04-19 16:04:49,630 - geodataframe.py - DEBUG - Next index: 43403
2023-04-19 16:04:49,631 - geodataframe.py - DEBUG - Next index: 43404
2023-04-19 16:04:49,632 - geodataframe.py - DEBUG - Next index: 43405
2023-04-19 16:04:49,

2023-04-19 16:04:49,735 - geodataframe.py - DEBUG - Next index: 43510
2023-04-19 16:04:49,736 - geodataframe.py - DEBUG - Next index: 43511
2023-04-19 16:04:49,737 - geodataframe.py - DEBUG - Next index: 43512
2023-04-19 16:04:49,738 - geodataframe.py - DEBUG - Next index: 43513
2023-04-19 16:04:49,739 - geodataframe.py - DEBUG - Next index: 43514
2023-04-19 16:04:49,740 - geodataframe.py - DEBUG - Next index: 43515
2023-04-19 16:04:49,741 - geodataframe.py - DEBUG - Next index: 43516
2023-04-19 16:04:49,742 - geodataframe.py - DEBUG - Next index: 43517
2023-04-19 16:04:49,743 - geodataframe.py - DEBUG - Next index: 43518
2023-04-19 16:04:49,743 - geodataframe.py - DEBUG - Next index: 43519
2023-04-19 16:04:49,744 - geodataframe.py - DEBUG - Next index: 43520
2023-04-19 16:04:49,745 - geodataframe.py - DEBUG - Next index: 43521
2023-04-19 16:04:49,746 - geodataframe.py - DEBUG - Next index: 43522
2023-04-19 16:04:49,747 - geodataframe.py - DEBUG - Next index: 43523
2023-04-19 16:04:49,

In [7]:
# read the basin shapefile and write gruId list
data = gpd.read_file(basin_gru_shp)
if not gru_fieldname in data.columns.values:
    exit(gru_fieldname + ' column does not exist in shapefile ' + basin_gru_shp)
else:
    grus = data[gru_fieldname].values
    
if 'int' in str(grus.dtype):
    np.savetxt(basin_gruId_txt, grus, fmt='%d')
else:
    np.savetxt(basin_gruId_txt, grus, fmt='%s')
print('wrote gruId file for the target basin %s: %s' % (basin_name, basin_gruId_txt))

In [8]:
# reproject basin GRU shapefile if it doesn't exist
if not os.path.exists(basin_gru_prj_shp):
    ga.reproject_vector(basin_gru_shp, basin_gru_prj_shp, new_epsg)
print('reprojected basin GRUs:', basin_gru_prj_shp)

# Alternative method: use ogr2ogr
#if not os.path.exists(basin_gru_prj_shp):
#    ga.reproject_basin_shapefile(basin_gru_shp, basin_gru_prj_shp, dst_crs)
#in_gdf_prj = gpd.read_file(basin_gru_prj_shp)    # read projected file in using geopandas

#### Extract basin flowline shapefile ####

In [9]:
# -- extract basin flowlines from full-dom flowlines file if it doesn't exist
#    note that the basin flowlines shapefile will be in the common projected coordinates (new_epsg)
#    this step can take a few minutes (wait for 'done')
if not os.path.exists(basin_flowlines_shp):
    
    # may need to reproject full-domain flowlines shapefile first
    flowlines_shp     = ut.read_from_control(control_file, 'fulldom_flowlines_shp')
    flowlines_prj_shp = flowlines_shp.split('.shp')[0]+'_prj.shp' 
    if not os.path.exists(flowlines_prj_shp):
        ga.reproject_vector(flowlines_shp, flowlines_prj_shp, new_epsg)
        print('reprojected full domain streams:', flowlines_prj_shp)
        
    # read stream and boundary files (projected)
    flowlines_gpd = gpd.read_file(flowlines_prj_shp)
    basin_gru_gpd = gpd.read_file(basin_gru_prj_shp)
    print('read reprojected shapefiles for clipping flowlines')    

    # create basin outer boundary shapefile 
    tmp_gpd                  = basin_gru_gpd[['geometry']]
    basin_gru_gpd['tmp_col'] = 0         # create null column for dissolve
    basin_boundary_gpd       = basin_gru_gpd.dissolve(by='tmp_col')
    basin_boundary_prj_shp   = basin_gru_prj_shp.split('.shp')[0]+'_boundary.shp'
    basin_boundary_gpd.to_file(basin_boundary_prj_shp)
    print('wrote basin boundary shapefile to use in stream clipping:', basin_boundary_prj_shp) 
    
    # clip full-dom reprojected flowlines with basin boundary     
    #   note: if geopandas version < 0.7, cannot use clip(), so instead use ogr2ogr
    if float(gpd.__version__.split(".")[0]+"."+gpd.__version__.split(".")[1]) >= 0.7:
        in_gpd_clip = gpd.clip(flowlines_gpd, basin_boundary_gpd)
        in_gpd_clip.to_file(basin_flowlines_shp)
    else:
        print('Note: using ogr2ogr to clip streams to basin')
        driverName = 'ESRI Shapefile'    # can later be upgraded to work with geopackages (eg 'GPKG')
        ogr2ogr.main(["", "-f", driverName, "-clipsrc", basin_boundary_prj_shp, basin_flowlines_shp, flowlines_prj_shp]) 
        
    print('wrote basin-clipped stream shapefile:', basin_flowlines_shp)
    print('done')